# Simulations sur les Chaînes de Markov à espace d'états fini

On commence par importer les bibliothèques usuelles :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Définitions

Définition de la chaîne de Markov à 5 états

In [ ]:
class MarkovChain:
    def __init__(self, transition_matrix):
        self.transition_matrix = transition_matrix
        self.num_states = transition_matrix.shape[0]

    def next_state(self, current_state):
        return np.random.choice(
            self.num_states,
            p=self.transition_matrix[current_state]
        )

Définition des vecteurs de déplacement associés à chaque état météo

In [ ]:
class SailingBoat:
    def __init__(self, markov_chain, vectors, noise_level):
        self.markov_chain = markov_chain
        self.vectors = vectors
        self.noise_level = noise_level
        self.position = np.array([0.0, 0.0])
        self.state = 0

    def move(self):
        self.state = self.markov_chain.next_state(self.state)
        vector = self.vectors[self.state]
        noise = np.random.normal(0, self.noise_level, size=vector.shape)
        self.position += vector + noise

    def get_position(self):
        return self.position
    
    def get_state(self):
        return self.state

    def reset_position(self):
        self.position = np.array([0.0, 0.0])

Définition des matrices de transition et des vecteurs de déplacement

In [ ]:
transition_matrix = np.array([
    [0.5, 0.3, 0.1, 0.05, 0.05],
    [0.2, 0.5, 0.2, 0.05, 0.05],
    [0.1, 0.2, 0.5, 0.1, 0.1],
    [0.05, 0.1, 0.2, 0.5, 0.15],
    [0.05, 0.05, 0.1, 0.2, 0.6]
])

vectors = [
    np.array([1.0, 0.0]),   # Beau temps
    np.array([0.5, 0.5]),   # Temps nuageux
    np.array([0.0, 1.0]),   # Pluie
    np.array([-0.5, 0.5]),  # Tempête
    np.array([-1.0, 0.0])   # Ouragan
]

noise_level = 0.1

Initialisation de la chaîne de Markov et du bateau

In [ ]:
markov_chain = MarkovChain(transition_matrix)
sailing_boat = SailingBoat(markov_chain, vectors, noise_level)

## 2 - Simulations

# Evolution du voilier sur le plan

In [ ]:

num_steps = 1000
positions = np.zeros((num_steps, 2))

for i in range(num_steps):
    sailing_boat.move()
    positions[i] = sailing_boat.get_position()

# Visualisation de l évolution du voilier
plt.figure(figsize=(10, 6))
plt.plot(positions[:, 0], positions[:, 1], marker='o', markersize=2, linestyle='-', alpha=0.6)
plt.title('Évolution du voilier sur le plan')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.show()

# Mesure Invariante

In [ ]:
def invariant_measure(transition_matrix, num_iterations=10000):
    num_states = transition_matrix.shape[0]
    state_counts = np.zeros(num_states)
    state = 0

    for _ in range(num_iterations):
        state_counts[state] += 1
        state = np.random.choice(num_states, p=transition_matrix[state])

    return state_counts / num_iterations

measure = invariant_measure(transition_matrix)
plt.figure(figsize=(10, 6))
plt.bar(range(5), measure, tick_label=['Beau', 'Nuageux', 'Pluie', 'Tempête', 'Ouragan'])
plt.title('Mesure invariante de la chaîne de Markov')
plt.xlabel('États météorologiques')
plt.ylabel('Probabilité')
plt.grid(True)
plt.show()

Voici une simulation permettant de comprendre l'utilité de la mesure invariante

In [ ]:
def empirical_distribution(sailing_boat, num_steps):
    state_counts = np.zeros(markov_chain.num_states)
    for _ in range(num_steps):
        sailing_boat.move()
        state_counts[sailing_boat.get_state()] += 1
    return state_counts / num_steps

num_steps_long = 100000
empirical_dist = empirical_distribution(SailingBoat(markov_chain, vectors, noise_level), num_steps_long)

plt.figure(figsize=(10, 6))
plt.bar(range(5), empirical_dist, tick_label=['Beau', 'Nuageux', 'Pluie', 'Tempête', 'Ouragan'], alpha=0.7, label='Distribution empirique')
plt.bar(range(5), measure, tick_label=['Beau', 'Nuageux', 'Pluie', 'Tempête', 'Ouragan'], alpha=0.7, label='Mesure invariante', color='orange')
plt.title('Comparaison entre la distribution empirique et la mesure invariante')
plt.xlabel('États météorologiques')
plt.ylabel('Probabilité')
plt.legend()
plt.grid(True)
plt.show()

# Théorème ergodique

In [ ]:
def ergodic_simulation(sailing_boat, num_steps):
    positions = np.zeros((num_steps, 2))
    for i in range(num_steps):
        sailing_boat.move()
        positions[i] = sailing_boat.get_position()
    return positions

# Paramètres pour les simulations ergodiques
num_simulations = 100
num_steps = 1000

# Effectuer plusieurs simulations ergodiques
ergodic_positions = [ergodic_simulation(SailingBoat(markov_chain, vectors, noise_level), num_steps) for _ in range(num_simulations)]

# Calculer les positions finales pour déterminer le rayon
final_positions = np.array([positions[-1] for positions in ergodic_positions])
center = np.mean(final_positions, axis=0)
distances = np.linalg.norm(final_positions - center, axis=1)
radius = np.percentile(distances, 95)

# Visualisation des simulations ergodiques avec le cercle de 95% de confiance
plt.figure(figsize=(10, 6))
for positions in ergodic_positions:
    plt.plot(positions[:, 0], positions[:, 1], marker='o', markersize=1, linestyle='-', alpha=0.6)
circle = plt.Circle(center, radius, color='r', fill=False, linestyle='--', linewidth=2, label='Cercle de 95% de confiance')
plt.gca().add_patch(circle)
plt.scatter(final_positions[:, 0], final_positions[:, 1], color='b', s=10, label='Positions finales')
plt.scatter(center[0], center[1], color='r', s=50, label='Centre de la distribution')
plt.title('Simulations du théorème ergodique avec cercle de 95% de confiance')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid(True)
plt.show()

# Théorème central limite

In [ ]:
def central_limit_simulation(sailing_boat, num_steps):
    for _ in range(num_steps):
        sailing_boat.move()
    return sailing_boat.get_position()

# Effectuer plusieurs simulations pour le théorème central limite
clt_samples = [central_limit_simulation(SailingBoat(markov_chain, vectors, noise_level), num_steps) for _ in range(1000)]
clt_samples = np.array(clt_samples)


Visualisation de l'histogramme et de la courbe gaussienne pour la coordonnée X

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(clt_samples[:, 0], bins=30, density=True, alpha=0.6, color='g', label='Histogramme des positions finales (X)')
mu, std = norm.fit(clt_samples[:, 0])
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2, label=f'Courbe normale ajustée (X) \n$\mu={mu:.2f}, \sigma={std:.2f}$')
plt.title('Histogramme et courbe gaussienne pour les positions finales (X)')
plt.xlabel('X')
plt.ylabel('Densité de probabilité')
plt.legend()
plt.grid(True)
plt.show()

Visualisation de l'histogramme et de la courbe gaussienne pour la coordonnée Y

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(clt_samples[:, 1], bins=30, density=True, alpha=0.6, color='b', label='Histogramme des positions finales (Y)')
mu, std = norm.fit(clt_samples[:, 1])
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2, label=f'Courbe normale ajustée (Y) \n$\mu={mu:.2f}, \sigma={std:.2f}$')
plt.title('Histogramme et courbe gaussienne pour les positions finales (Y)')
plt.xlabel('Y')
plt.ylabel('Densité de probabilité')
plt.legend()
plt.grid(True)
plt.show()

# Temps nécessaire pour accoster en un certain point

In [ ]:
def time_to_destination(sailing_boat, destination, tolerance=0.1):
    time_steps = 0
    while np.linalg.norm(sailing_boat.get_position() - destination) > tolerance:
        sailing_boat.move()
        time_steps += 1
    return time_steps

destination = np.array([10.0, 10.0])
num_simulations = 1000
times_to_destination = [time_to_destination(SailingBoat(markov_chain, vectors, noise_level), destination) for _ in range(num_simulations)]

# Visualisation du temps nécessaire pour accoster en un certain point
plt.figure(figsize=(10, 6))
plt.hist(times_to_destination, bins=30, density=True, alpha=0.6, color='m', label='Histogramme du temps nécessaire pour accoster')
mu, std = norm.fit(times_to_destination)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2, label=f'Courbe normale ajustée \n$\mu={mu:.2f}, \sigma={std:.2f}$')
plt.title('Histogramme du temps nécessaire pour accoster en un certain point')
plt.xlabel('Temps (nombre de pas)')
plt.ylabel('Densité de probabilité')
plt.legend()
plt.grid(True)
plt.show()

# Temps nécessaire pour revenir au point de départ

In [ ]:
def time_to_return(sailing_boat, max_distance, tolerance=0.1):
    initial_position = sailing_boat.get_position()
    time_steps = 0
    while np.linalg.norm(sailing_boat.get_position() - initial_position) < max_distance:
        sailing_boat.move()
    while np.linalg.norm(sailing_boat.get_position()) > tolerance:
        sailing_boat.move()
        time_steps += 1
    return time_steps

max_distance = 10.0
num_simulations = 1000
times_to_return = [time_to_return(SailingBoat(markov_chain, vectors, noise_level), max_distance) for _ in range(num_simulations)]

# Visualisation du temps nécessaire pour revenir au point de départ
plt.figure(figsize=(10, 6))
plt.hist(times_to_return, bins=30, density=True, alpha=0.6, color='c', label='Histogramme du temps nécessaire pour revenir au point de départ')
mu, std = norm.fit(times_to_return)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2, label=f'Courbe normale ajustée \n$\mu={mu:.2f}, \sigma={std:.2f}$')
plt.title('Histogramme du temps nécessaire pour revenir au point de départ')
plt.xlabel('Temps (nombre de pas)')
plt.ylabel('Densité de probabilité')
plt.legend()
plt.grid(True)
plt.show()

$\mathcal{FIN}$.